In [12]:
import urllib.request
import time
import json
nextBikeUrl = 'https://api.nextbike.net/maps/nextbike-live.json?city=14'

In [13]:
page = urllib.request.urlopen(nextBikeUrl)
data = page.read()

"""Process the crawled information."""
dataString = data.decode('utf8').replace("'", '"')
dataJson = json.loads(dataString)

In [14]:
bikedata = dataJson["countries"][0]["cities"][0]["places"]

In [15]:
bikedata[1]

{'address': None,
 'bike': False,
 'bike_list': [],
 'bike_numbers': [],
 'bike_racks': 0,
 'bike_types': [],
 'bikes': 0,
 'free_racks': 0,
 'lat': 50.9429485,
 'lng': 6.958015,
 'maintenance': False,
 'name': 'Hauptbahnhof',
 'number': 4826,
 'place_type': '0',
 'rack_locks': False,
 'spot': True,
 'terminal_type': 'free',
 'uid': 35409}

In [16]:
bikes = []
stations = []
for i in bikedata:
    if i["bike"] is True:
        bikes.append(i)
    else:
        stations.append(i)

print("Räder", len(bikes))
print("Stationen", len(stations))

Räder 643
Stationen 21


In [17]:
for i in stations:
    print(i["name"])

Bahnhof Mühlheim 
Hauptbahnhof
KVB Hauptverwaltung
Zülpicher Platz
Neumarkt
Uni Köln
Maritim 
Ebertplatz
Hans-Böckler-Platz
Bahnhof Deutz
Breslauer Platz
Severinsstrasse
Wienerplatz
RTL Picassoplatz
Uni Mensa
Chlodwigplatz
TH Köln Südstadt
Leonardo Royal Hotel
Hotel Stadtpalais
Motel One Mediapark
Motel One Waidmarkt
